In [1]:
import seaborn as sns
sns.set_style("whitegrid")
from process_model_nosim_cont import *
from fit_process import *

from scipy.stats import norm

#from chase.base import *
#from chase.utils import *
#from chase.fit import *

import pickle

PARS = {'theta': [0, 40, 2],
        'p_stay': [0, 1, .5],
        'tau': [0, 1, .5],
        'p_fixed': [0, 1, .5],
        'p_stay': [0, 1, .5],
        'p_switch': [0, 1, .5],
        'prelec_gamma': [0, 5, 1.],
        'prelec_elevation': [0, 5, 1.],
        'prelec_gamma_loss': [0, 5, 1.],
        'prelec_elevation_loss': [0, 5, 1.],
        'pow_gain': [0., np.inf, 1.],
        'w_loss': [0., np.inf, 1.],
        'phi': [-np.inf, np.inf, 0.],}

#FIGDEST = '/Users/markant/Dropbox/Apps/ShareLaTeX/ssDFE/ssDFE_v3/figures/'

def convert_to_fit(N_gen, problems, results):
    data = []
    for pid in problems:
        for i in range(N_gen):
            data.append({'group': 0, 
                         'probid': pid,
                         'sampled_option': results[pid]['sampled_option'][i],
                         'outcomes': results[pid]['outcomes'][i],
                         'choice': results[pid]['choice'][i]})
    return data

# Recovery - six problems

In [2]:
problems = {}
arr = np.genfromtxt('paper/data/six_problems.csv', delimiter=',')
for i in range(len(arr)):
    problems[i] = arr[i].reshape((2,2,2))

### Recover theta, tau

In [43]:
N_gen = 100
FIXED = {'minsamplesize': 2}
PARSETS = [['theta', 'tau', 'p_switch']]
N_ITER = 1
SIM_ID = 'recover_process'
OUTDIR = 'process_fitresults'

fitresults = {}

for theta in [1, 3, 5]:
    for tau in [.01, .5, .99]:

        print 'theta: %s' % theta
        print 'tau: %s' % tau
        k = (theta, tau)
        
        # first generate some data
        print 'generating data..'
        results = {}    
        m = CHASEProcessModel(problems=problems,
                              startdist='uniform')
        for pid in problems:
            pars = {'probid': pid,
                    'minsamplesize': 2,
                    'theta': theta,
                    'tau': tau,
                    'p_switch': .3,
                    'max_T': 1000,
                    'N': N_gen}
            results[pid] = m(problems[pid], pars)

        fitdata = convert_to_fit(N_gen, problems, results)            

        for parset in PARSETS:

            fitting = {p: PARS[p] for p in parset}

            fitresults[k] = fit_mlh(m, problems, fitdata, 
                                    SIM_ID, FIXED, fitting, niter=N_ITER, outdir=OUTDIR)
            print fitresults[k]

theta: 1
tau: 0.01
generating data..
p_switch	tau	theta
[ 0.5  0.5  2. ] 2843.29389564 5.16871213913
[ 0.525  0.5    2.   ] 2908.69507867 5.5264749527
[ 0.5    0.525  2.   ] 2802.4376023 5.53130483627
[ 0.5  0.5  2.1] 2938.91748451 5.76059603691
[ 0.517  0.517  1.9  ] 2793.68491314 5.52794289589
[ 0.525  0.525  1.8  ] 2768.5485658 5.41242098808
[ 0.492  0.533  1.867] 2711.03589742 5.58577799797
[ 0.475  0.55   1.8  ] 2642.44126858 5.90004110336
[ 0.5    0.567  1.733] 2673.15581694 6.24110913277
[ 0.5    0.569  1.556] 2607.87644246 5.65750789642
[ 0.5    0.592  1.333] 2542.07125796 7.21095013618
[ 0.458  0.614  1.444] 2490.38053995 5.3617401123
[ 0.425  0.658  1.267] 2401.84474231 5.09569406509
[ 0.433  0.633  1.2  ] 2385.29476761 4.64239215851
[ 0.4    0.667  0.933] 2278.92152892 4.17031598091
[ 0.408  0.728  0.556] 2648.09156811 3.37433385849
[ 0.458  0.594  1.489] 2497.57511756 5.12704920769
[ 0.356  0.688  1.126] 2295.67662736 4.75501918793
[ 0.329  0.748  0.728] 2213.54111344 3.918

In [184]:
with open('process_fitresults/recover_tau_theta.pkl', 'r') as f:
    fitresults = pickle.load(f)

In [215]:
#with open('process_fitresults/recover_tau_theta.pkl', 'w') as f:
#    pickle.dump(fitresults, f)

In [216]:
arr = []
for theta in [1, 3, 5]:
    for tau in [.01, .5, .99]:
        k = (theta, tau)
        r = fitresults[k]
        arr.append([theta, tau, fitresults[k]['theta'].values[0], fitresults[k]['tau'].values[0]])
df = pd.DataFrame(arr, columns=['gen_theta', 'gen_tau', 'bf_theta', 'bf_tau'])
df.to_csv('process_fitresults/recover_tau_theta.csv')

In [217]:
df

,gen_theta,gen_tau,bf_theta,bf_tau
0,1,0.01,1.085212,0.067770
1,1,0.50,1.008781,0.509048
2,1,0.99,0.999557,0.997573
3,3,0.01,2.994716,0.011590
4,3,0.50,3.004007,0.508795
5,3,0.99,3.018064,0.989163
6,5,0.01,5.005642,0.006260
7,5,0.50,4.993989,0.499533
8,5,0.99,5.001797,0.993472


### Recover theta, tau, gamma

In [ ]:
N_gen = 100
FIXED = {'minsamplesize': 2}
PARSETS = [['theta', 'tau', 'prelec_gamma']]
N_ITER = 1
SIM_ID = 'recover_process'
OUTDIR = 'process_fitresults'

fitresults = {}

for theta in [1, 3, 5]:
    for tau in [.01, .5, .99]:
        for prelec_gamma in [.7, 1, 1.3]:

            print 'theta: %s' % theta
            print 'tau: %s' % tau
            print 'gamma: %s' % prelec_gamma
            k = (theta, tau, prelec_gamma)

            # first generate some data
            print 'generating data..'
            results = {}    
            m = CHASEProcessModel(problems=problems,
                                  startdist='uniform')
            for pid in problems:
                pars = {'probid': pid,
                        'minsamplesize': 2,
                        'theta': theta,
                        'tau': tau,
                        'prelec_gamma': prelec_gamma,
                        'max_T': 1000,
                        'N': N_gen}
                results[pid] = m(problems[pid], pars)

            fitdata = convert_to_fit(N_gen, problems, results)            

            for parset in PARSETS:

                fitting = {p: PARS[p] for p in parset}

                fitresults[k] = fit_mlh(m, problems, fitdata, 
                                        SIM_ID, FIXED, fitting, niter=N_ITER, outdir=OUTDIR)
                print fitresults[k]

In [219]:
#with open('process_fitresults/recover_tau_theta_gamma.pkl', 'w') as f:
#    pickle.dump(fitresults, f)

In [220]:
arr = []
for theta in [1, 3, 5]:
    for tau in [.01, .5, .99]:
        for prelec_gamma in [.7, 1, 1.3]:
            k = (theta, tau, prelec_gamma)
            r = fitresults[k]
            arr.append([theta, tau, prelec_gamma,
                        fitresults[k]['theta'].values[0], 
                        fitresults[k]['tau'].values[0],
                        fitresults[k]['prelec_gamma'].values[0]])
df = pd.DataFrame(arr, columns=['gen_theta', 'gen_tau', 'gen_gamma', 'bf_theta', 'bf_tau', 'bf_gamma'])
df.to_csv('process_fitresults/recover_tau_theta_gamma.csv')

In [221]:
df

,gen_theta,gen_tau,gen_gamma,bf_theta,bf_tau,bf_gamma
0,1,0.01,0.7,0.940369,0.071466,0.646968
1,1,0.01,1.0,1.067589,0.556170,1.232704
2,1,0.01,1.3,1.078219,0.005863,1.331694
3,1,0.50,0.7,1.054392,0.431949,0.737681
4,1,0.50,1.0,1.001585,0.509881,0.992045
5,1,0.50,1.3,1.011063,0.451068,1.314481
6,1,0.99,0.7,0.974922,0.982321,0.661906
7,1,0.99,1.0,1.025482,0.946213,1.009916
8,1,0.99,1.3,1.014345,0.998229,1.295270
9,3,0.01,0.7,3.008939,0.023305,0.711126


### Recover theta, tau, gamma+delta

In [3]:
N_gen = 100
FIXED = {'minsamplesize': 2}
PARSETS = [['theta', 'tau', 'prelec_gamma', 'prelec_elevation']]
N_ITER = 1
SIM_ID = 'recover_process'
OUTDIR = 'process_fitresults'

fitresults = {}

for theta in [3]:
    for tau in [.01, .5]:
        for prelec_gamma in [.7, 1, 1.3]:
            for prelec_elevation in [.7, 1, 1.3]:

                print 'theta: %s' % theta
                print 'tau: %s' % tau
                print 'gamma: %s' % prelec_gamma
                print 'delta: %s' % prelec_elevation
                k = (theta, tau, prelec_gamma, prelec_elevation)

                # first generate some data
                print 'generating data..'
                results = {}    
                m = CHASEProcessModel(problems=problems,
                                      startdist='uniform')
                for pid in problems:
                    pars = {'probid': pid,
                            'minsamplesize': 2,
                            'theta': theta,
                            'tau': tau,
                            'prelec_gamma': prelec_gamma,
                            'prelec_elevation': prelec_elevation,
                            'max_T': 1000,
                            'N': N_gen}
                    results[pid] = m(problems[pid], pars)

                fitdata = convert_to_fit(N_gen, problems, results)            

                for parset in PARSETS:

                    fitting = {p: PARS[p] for p in parset}

                    fitresults[k] = fit_mlh(m, problems, fitdata, 
                                            SIM_ID, FIXED, fitting, niter=N_ITER, outdir=OUTDIR)
                    print fitresults[k]

cpt.py:255: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  weights[gaindf.iloc[i]['id']] = gaindf.iloc[i]['w']
cpt.py:258: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  weights[lossdf.iloc[i]['id']] = lossdf.iloc[i]['w']


theta: 3
tau: 0.01
gamma: 0.7
delta: 0.7
generating data..
prelec_elevation	prelec_gamma	tau	theta
[ 1.   1.   0.5  2. ] 21958.5510533 6.16112494469
[ 1.05  1.    0.5   2.  ] 22253.0844486 6.23390388489
[ 1.    1.05  0.5   2.  ] 27081.6831392 6.36274409294
[ 1.     1.     0.525  2.   ] 21920.5131796 7.01422786713
[ 1.   1.   0.5  2.1] 18869.5102666 7.05445194244
[ 1.025  0.95   0.512  2.05 ] 18692.7177509 7.0509018898
[ 1.037  0.9    0.519  2.075] 13987.5283608 6.60559391975
[ 0.969  0.95   0.522  2.087] 18383.2558428 6.66710495949
[ 1.003  0.925  0.533  2.131] 12450.0174432 6.87813305855
[ 1.005  0.887  0.549  2.197] 7036.45862135 7.78277802467
[ 1.005  0.869  0.52   2.23 ] 6902.8385999 8.00738406181
[ 1.008  0.803  0.517  2.345] 3681.20365529 8.34046506882
[ 1.01   0.77   0.554  2.252] 3611.81893129 8.85555911064
[ 1.014  0.655  0.58   2.328] 1205.78535707 10.9504070282
[ 1.064  0.673  0.561  2.385] 1437.86200707 10.3332169056
[ 1.008  0.61   0.585  2.552] 721.439910235 10.7761321068

In [4]:
#with open('process_fitresults/recover_tau_theta_gamma_delta.pkl', 'w') as f:
#    pickle.dump(fitresults, f)

In [5]:
arr = []
for theta in [3]:
    for tau in [.01, .5]:
        for prelec_gamma in [.7, 1, 1.3]:
            for prelec_elevation in [.7, 1, 1.3]:

                k = (theta, tau, prelec_gamma, prelec_elevation)
                r = fitresults[k]
                arr.append([theta, tau, prelec_gamma, prelec_elevation,
                            fitresults[k]['theta'].values[0], 
                            fitresults[k]['tau'].values[0],
                            fitresults[k]['prelec_gamma'].values[0],
                            fitresults[k]['prelec_elevation'].values[0]])
df = pd.DataFrame(arr, columns=['gen_theta', 'gen_tau', 'gen_gamma', 'gen_delta', 
                                'bf_theta', 'bf_tau', 'bf_gamma', 'bf_delta'])
df.to_csv('process_fitresults/recover_tau_theta_gamma_delta.csv')

In [6]:
df

,gen_theta,gen_tau,gen_gamma,gen_delta,bf_theta,bf_tau,bf_gamma,bf_delta
0,3,0.01,0.7,0.7,3.181363,0.079489,0.707805,0.706263
1,3,0.01,0.7,1.0,2.945587,0.119801,0.714265,1.010159
2,3,0.01,0.7,1.3,3.004747,0.013568,0.690723,1.310913
3,3,0.01,1.0,0.7,2.965379,0.016047,0.975958,0.710066
4,3,0.01,1.0,1.0,2.991314,0.357327,0.850808,1.139071
5,3,0.01,1.0,1.3,3.001410,0.008094,1.000204,1.299613
6,3,0.01,1.3,0.7,3.005484,0.030949,1.243542,0.732908
7,3,0.01,1.3,1.0,3.008083,0.011722,1.301786,0.997805
8,3,0.01,1.3,1.3,3.000877,0.024665,1.302244,1.300516
9,3,0.50,0.7,0.7,2.944497,0.498093,0.689250,0.701738


In [9]:
np.sum(np.log([.5, .2]))

-2.3025850929940455

In [8]:
np.log(.5*.2)

-2.3025850929940455

## p-stay

In [4]:
from process_model_nosim_cont_pstay import *

In [40]:
N_gen = 20
FIXED = {'minsamplesize': 2,
         'p_switch': .25}
PARSETS = [['theta', 'tau', 'p_stay']]
N_ITER = 1
SIM_ID = 'recover_process'
OUTDIR = 'process_fitresults'

fitresults = {}

for theta in [3]:
    for tau in [.1]:
        for p_stay in [.01, .2, .5]:

            print 'theta: %s' % theta
            print 'tau: %s' % tau
            print 'p_stay: %s' % p_stay
            k = (theta, tau, p_stay)

            # first generate some data
            print 'generating data..'
            results = {}    
            m = CHASEProcessModel(problems=problems,
                                  startdist='uniform')
            for pid in problems:
                pars = {'probid': pid,
                        'minsamplesize': 2,
                        'theta': theta,
                        'tau': tau,
                        'p_stay': p_stay,
                        'p_switch': .25,
                        'max_T': 1000,
                        'N': N_gen}
                results[pid] = m(problems[pid], pars)

            fitdata = convert_to_fit(N_gen, problems, results)            

            for parset in PARSETS:

                fitting = {p: PARS[p] for p in parset}

                fitresults[k] = fit_mlh(m, problems, fitdata, 
                                        SIM_ID, FIXED, fitting, 
                                        niter=N_ITER, ftol=.1, outdir=OUTDIR)
                print fitresults[k]

theta: 3
tau: 0.1
p_stay: 0.01
generating data..
p_stay	tau	theta
[ 0.5  0.5  2. ] 195.028860394 15.5482449532
[ 0.525  0.5    2.   ] 198.759247838 14.9539561272
[ 0.5    0.525  2.   ] 195.36181143 15.0399770737
[ 0.5  0.5  2.1] 195.150761436 15.9378111362
[ 0.475  0.517  2.067] 191.460364596 14.8932549953
[ 0.45   0.525  2.1  ] 187.797562865 14.9292461872
[ 0.467  0.492  2.133] 189.336375521 14.8887169361
[ 0.444  0.511  2.056] 187.34293453 15.3603019714
[ 0.417  0.517  2.033] 184.219552324 14.9440281391
[ 0.389  0.522  2.178] 177.769119708 15.0897381306
[ 0.333  0.533  2.267] 168.035166994 14.8934819698
[ 0.333  0.558  2.133] 171.568137406 14.9196248055
[ 0.272  0.547  2.189] 160.184783241 14.8434960842
[ 0.183  0.558  2.233] 145.740773441 14.593416214
[ 0.15   0.583  2.389] 136.545613186 14.4338481426
[ 0.017  0.617  2.567] 121.092237959 13.7177791595
[ 0.022  0.581  2.578] 116.465047716 13.6979429722
[ 0.204  0.559  2.363] 144.389161591 14.6392011642
[ 0.132  0.572  2.368] 133.8818

### Mixture of optional and fixed stopping

In [34]:
N_gen = 100
FIXED = {'minsamplesize': 2}
PARSETS = [['theta', 'tau', 'p_fixed']]
N_ITER = 1
SIM_ID = 'recover_process'
OUTDIR = 'process_fitresults'

fitresults = {}

for theta in [3]:
    for tau in [.01, .5, .99]:
        for p_fixed in [.3, .6]:

            print 'theta: %s' % theta
            print 'tau: %s' % tau
            print 'p_fixed: %s' % p_fixed
            k = (theta, tau, p_fixed)

            # first generate some data
            print 'generating data..'
            results = {}    
            m = CHASEProcessModel(problems=problems,
                                  startdist='uniform')
            for pid in problems:
                pars = {'probid': pid,
                        'minsamplesize': 2,
                        'theta': theta,
                        'tau': tau,
                        'p_fixed': p_fixed,
                        'max_T': 1000,
                        'N': N_gen}
                results[pid] = m(problems[pid], pars)

            fitdata = convert_to_fit(N_gen, problems, results)            

            for parset in PARSETS:

                fitting = {p: PARS[p] for p in parset}

                fitresults[k] = fit_mlh(m, problems, fitdata, 
                                        SIM_ID, FIXED, fitting, niter=N_ITER, outdir=OUTDIR)
                print fitresults[k]

theta: 3
tau: 0.01
p_fixed: 0.3
generating data..


IndexError: index -9223372036854775808 is out of bounds for axis 1 with size 1000